In [ ]:
import ct.mlexp2 as ml
from importlib import reload
from datetime import date
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.data.glassnode.utils import get_glassnode_data, quantile_info, url_parser
import time
from IPython.display import Audio
from cdcqr.common.config import GLASSNODE_API_KEY
sound_file = '/tmp/ding.wav'
import sys
sys.path.append('/github/cryptoderiv-quant/')
from ct.utils import glassnode

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import pandas as pd
from ct.fs import runscoring

### prepare data

In [ ]:
data_sym = 'BTCUSD_PERP@binance'
df1 = load_df(data_sym).resample_pv('10Min')
df1.head()

In [ ]:
category = 'Transactions'
feature_name ='TransfersVolumeToExchangesSum'

underly = 'BTC'
url = 'a={}&category={}&m=transactions.{}'.format(underly, category, feature_name)
att_dict = url_parser(url)
category = att_dict['category']
feature = att_dict['feature']
underly = att_dict['a']

freq='10m'
start_time = df1.index[0].isoformat()
url1 = 'metrics~{}~{}'.format(category, feature)
dfgsi = glassnode(url1,a=underly,i=freq,s=start_time,price=False)
df1 = df1[df1.index<='20211031']
dff = dfgsi.reindex(df1.index)

In [ ]:
df_feature = dff.copy()
df0 = df1.copy()
sym = 'BTC'
qsuffix='USDT@binancealt'
ytrain_thresh=6
ytrain_periods=100
ytest_thresh=3
ytest_periods=100
vdelay=2
ytypes=["ys"]
xtypes=["v"]
qlags=1000
request=['vbt','vbtplot','plottree','showsplit']
#models=["oneiflarger","dt"]
models=["oneiflarger", 'rf']
randcv_n_iter=30
sl=2
tp=6
ts=None
vbtfreq="10Min"

In [ ]:
res=[]
    
sym=sym                 
t0 = time.time()
print('adding ysbarrier')
y=df0.addysbarrier(thresh=ytrain_thresh,periods=ytrain_periods,inplace=True,retfnames=True)[0]
print('addysbarrier done 1/2', time.time()-t0)
yt=df0.addysbarrier(thresh=ytest_thresh,periods=ytest_periods,inplace=True,retfnames=True)[0]
print('addysbarrier done 2/2', time.time()-t0)


df=df0.drop(columns=['v']).join(df_feature)
df['vd']=df['v'].shift(vdelay)

In [ ]:
df.head()

In [ ]:
allxs=['vd']
vdmed=df.vd.median()
df['vdm']=df['vd']
df.loc[df.vd<df.vd.median(),'vdm']=0

In [ ]:
x = 'vd'
xs=[x]
dftrain=df.iloc[:int(0.7*len(df))].dropna()
dftest=df.iloc[-int(0.2*len(df)):].dropna()

In [ ]:
cv=TimeSeriesSplit(gap=100, max_train_size=len(dftrain)//2,n_splits=3, test_size=len(dftrain)//4) #1min 30 secs


In [ ]:
if df[y].value_counts(normalize=True).prod()<0.2:
    print('consider using classweight')

In [ ]:
modelstr = 'dummy'
modeld=ml.modelsdict[modelstr]
model=modeld['model']
model.set_params(**modeld['fixedparams'])
fit_params={}#'sample_weight':sample_weight}

randcv = RandomizedSearchCV(model, param_distributions=modeld['paramsd'], n_iter=randcv_n_iter, scoring='precision', n_jobs=10, cv=cv, verbose=0, random_state=1,refit=True).fit(dftrain[xs], dftrain[y],**fit_params)
display(pd.DataFrame(randcv.cv_results_).sort_values(by='mean_test_score',ascending=False)[['mean_test_score' ,'std_test_score', 'params','split0_test_score','split1_test_score','split2_test_score']])
model.set_params(**randcv.best_params_)
meancvscore=randcv.best_score_
print(model)
resscor=runscoring(df=dftrain,y=y,yt=yt,xs=xs,model=model,nansy='.fillna(0)',nansx=None,scoring='precision',verbose=100,cv=cv,dftest=dftest,request=request,sl=sl,tp=tp,ts=ts,vbtfreq=vbtfreq)


In [ ]:
modelstr ='oneiflarger'
modeld=ml.modelsdict[modelstr]
model=modeld['model']
model.set_params(**modeld['fixedparams'])
fit_params={}#'sample_weight':sample_weight}

randcv = RandomizedSearchCV(model, param_distributions=modeld['paramsd'], n_iter=randcv_n_iter, scoring='precision', n_jobs=10, cv=cv, verbose=0, random_state=1,refit=True).fit(dftrain[xs], dftrain[y],**fit_params)
display(pd.DataFrame(randcv.cv_results_).sort_values(by='mean_test_score',ascending=False)[['mean_test_score' ,'std_test_score', 'params','split0_test_score','split1_test_score','split2_test_score']])
model.set_params(**randcv.best_params_)
meancvscore=randcv.best_score_
print(model)
resscor=runscoring(df=dftrain,y=y,yt=yt,xs=xs,model=model,nansy='.fillna(0)',nansx=None,scoring='precision',verbose=100,cv=cv,dftest=dftest,request=request,sl=sl,tp=tp,ts=ts,vbtfreq=vbtfreq)


In [ ]:
modelstr ='rf'
modeld=ml.modelsdict[modelstr]
model=modeld['model']
model.set_params(**modeld['fixedparams'])
fit_params={}#'sample_weight':sample_weight}

randcv = RandomizedSearchCV(model, param_distributions=modeld['paramsd'], n_iter=randcv_n_iter, scoring='precision', n_jobs=10, cv=cv, verbose=0, random_state=1,refit=True).fit(dftrain[xs], dftrain[y],**fit_params)
display(pd.DataFrame(randcv.cv_results_).sort_values(by='mean_test_score',ascending=False)[['mean_test_score' ,'std_test_score', 'params','split0_test_score','split1_test_score','split2_test_score']])
model.set_params(**randcv.best_params_)
meancvscore=randcv.best_score_
print(model)
resscor=runscoring(df=dftrain,y=y,yt=yt,xs=xs,model=model,nansy='.fillna(0)',nansx=None,scoring='precision',verbose=100,cv=cv,dftest=dftest,request=request,sl=sl,tp=tp,ts=ts,vbtfreq=vbtfreq)


In [ ]:
modelstr ='dt'
modeld=ml.modelsdict[modelstr]
model=modeld['model']
model.set_params(**modeld['fixedparams'])
fit_params={}#'sample_weight':sample_weight}

randcv = RandomizedSearchCV(model, param_distributions=modeld['paramsd'], n_iter=randcv_n_iter, scoring='precision', n_jobs=10, cv=cv, verbose=0, random_state=1,refit=True).fit(dftrain[xs], dftrain[y],**fit_params)
display(pd.DataFrame(randcv.cv_results_).sort_values(by='mean_test_score',ascending=False)[['mean_test_score' ,'std_test_score', 'params','split0_test_score','split1_test_score','split2_test_score']])
model.set_params(**randcv.best_params_)
meancvscore=randcv.best_score_
print(model)
resscor=runscoring(df=dftrain,y=y,yt=yt,xs=xs,model=model,nansy='.fillna(0)',nansx=None,scoring='precision',verbose=100,cv=cv,dftest=dftest,request=request,sl=sl,tp=tp,ts=ts,vbtfreq=vbtfreq)
